# Analyse sentimentale avec Scikit-Learn

## Chargement de données

In [41]:
import sklearn

Depuis NLTK, on télécharge et importe un corpus contenant des commentaires sur des films et qui sont annotés; c'est à dire pour chaque commentaire on a l'info (sous forme de flag  ou 1) si c'est positif ou négatif...

In [1]:
from nltk.corpus import movie_reviews

On effectue une petite transformation (Reshaping) de données pour les préparer en entrée de notre algorithme de classification...

In [2]:
movie_train = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories() for fileid in movie_reviews.fileids(category)]

Un petit <b> shuffle </b>  est nécessaire pour que, plus tard,  on ait un entraînement et un test qui ne dépendent pas de la séquence de ces commentaires...

In [3]:
import random
random.shuffle(movie_train)

Notre dataset contient 2000 commentaires

In [4]:
len(movie_train)

2000

In [5]:
movie_train_data = [" ".join(item[0]) for item in movie_train]

In [6]:
movie_train_targets = [item[1] for item in movie_train]

## Numérisation des commentaires

On va numériser les commentaires en utilisant le procédé TF-IDF. On commence par calculer la fréquence <b> TF </b> de chaque mot dans chaque commentaire, puis on pondère ces fréquences par le terme <b> IDF </b>

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [17]:
from nltk import word_tokenize

In [18]:
movie_vec = CountVectorizer(min_df=2, tokenizer=word_tokenize)
movie_counts = movie_vec.fit_transform(movie_train_data)

In [22]:
tfidf_transformer = TfidfTransformer()
movie_tfidf = tfidf_transformer.fit_transform(movie_counts)

In [24]:
# Décommenter pour afficher les poids TF-IDF résultants
# movie_tfidf.toarray()

In [25]:
movie_tfidf.shape

(2000, 24160)

En sortie de TF-IDF, on a une dimensionnalité de 24160, qui est très élevée. On réduit celle là grâce à l'algo PCA (avec dimensionnalité cible égale à 300) ...

In [26]:
from sklearn.decomposition import PCA, TruncatedSVD
pca_model = TruncatedSVD(n_components = 300)

In [28]:
import pandas as pd

In [29]:
pca_features = pd.DataFrame(pca_model.fit_transform(movie_tfidf))

In [30]:
pca_features.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.616673,0.187617,0.020689,-0.010779,-0.043433,-0.046580,0.029482,-0.080083,0.014084,-0.023268,...,-0.016434,-0.002631,0.007213,-0.029626,0.002070,0.007734,0.019336,-0.003916,0.016984,0.000502
1,0.550033,-0.010215,-0.016043,-0.047319,-0.032550,0.005165,0.014986,-0.021206,0.022857,-0.016931,...,-0.000965,0.008323,0.014166,0.017873,-0.010285,-0.013008,0.027376,0.009148,0.042732,-0.034176
2,0.563653,-0.095314,-0.052334,0.137567,-0.036462,0.078363,0.063313,0.023514,0.032131,-0.034066,...,0.004594,0.055028,0.045293,-0.004875,0.003672,0.010198,-0.012674,0.049590,-0.031206,0.001216
3,0.579325,-0.064473,-0.101694,-0.060615,0.073223,-0.115141,0.039840,-0.022482,0.044138,-0.012617,...,-0.023026,-0.001419,0.017665,0.010269,0.014328,-0.003456,-0.000146,0.007905,-0.000146,0.001501
4,0.570528,-0.113899,0.007387,0.017383,0.027013,-0.058169,-0.055419,-0.010109,-0.058541,-0.026143,...,-0.032448,0.008412,0.010092,0.003922,0.000785,-0.020864,-0.018297,-0.024874,-0.007952,0.005767


On importe le modèle de SVM linéaire depuis Scikit-Learn

In [36]:
from sklearn.svm import LinearSVC

On effectue le Train Test Split en attribuant 20% des échantillons choisis aléatoirement à l'ensemble de test et le reste (80%) à l'ensemble de Train (apprentissage)

In [37]:
from sklearn.model_selection import train_test_split
docs_train, docs_test, y_train, y_test = train_test_split(
    pca_features, movie_train_targets, test_size = 0.20, random_state = 12)

On entraîne le modèle SVM linéaire sur les données d'apprentissage

In [39]:
clf = LinearSVC().fit(docs_train, y_train)

On effectue des prédictions sur tout l'ensemble de test et on calcule un score de précision (accuracy) comme étant le rapport du nombre de bonnes prédictions sur la taille de l'ensemble <br>
Ce calcul s'effectue grâce à la fonction <b> sklearn.metrics.accuracy_score </b> 

In [42]:
y_pred = clf.predict(docs_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.85

On peut aussi calculer une matrice de confusion à base de ces prédictions sur l'ensemble de test

In [28]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[169,  30],
       [ 31, 170]])